In [1]:
import os

In [2]:
%pwd

'd:\\SAMITH\\Github\\Heart_Rate_Anomaly_Detector\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\SAMITH\\Github\\Heart_Rate_Anomaly_Detector'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_heart_rate_data_path: Path
    test_heart_rate_data_path: Path
    train_is_anomaly_data_path: Path
    test_is_anomaly_data_path: Path
    data_transformation_dir: Path  
    
   
    heart_rate_predictor_model_name: str
    anomaly_detector_model_name: str
    
    
    heart_rate_target_column: str
    anomaly_target_column: str

In [7]:
from Heart_Rate_Anomaly_Detector.constants import *
from Heart_Rate_Anomaly_Detector.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        
        
        heart_rate_params = self.params.HEART_RATE_PREDICTOR
        anomaly_params = self.params.ANOMALY_DETECTOR
        
       
        heart_rate_model_schema = self.schema.models.HeartRatePredictor
        anomaly_model_schema = self.schema.models.AnomalyDetector

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_heart_rate_data_path=Path(config.data_path.train_heart_rate),
            test_heart_rate_data_path=Path(config.data_path.test_heart_rate),
            train_is_anomaly_data_path=Path(config.data_path.train_is_anomaly),
            test_is_anomaly_data_path=Path(config.data_path.test_is_anomaly),
            data_transformation_dir=Path(self.config.data_transformation.root_dir),
            
         
            heart_rate_predictor_model_name=config.model_name.heart_rate_predictor,
            anomaly_detector_model_name=config.model_name.anomaly_detector,
            
           
            heart_rate_target_column=heart_rate_model_schema.target_column,
            anomaly_target_column=anomaly_model_schema.target_column
        )
        return model_trainer_config
    
    def get_heart_rate_features(self) -> list:
       
        return self.schema.models.HeartRatePredictor.features
    
    def get_anomaly_features(self) -> list:
       
        return self.schema.models.AnomalyDetector.features
    
    def get_column_dtypes(self) -> dict:
        
        return self.schema.columns

In [ ]:
class ModelTrainer:
    def __init__(self, config):
        self.config = config
        self.scaler = None
        self.feature_columns = None


    def heart_rate_load_scaler(self):
        
        scaler_path = os.path.join(self.config.data_transformation_dir, "scaler.joblib")
        if os.path.exists(scaler_path):
            self.scaler = joblib.load(scaler_path)
            logger.info(f"Scaler loaded from: {scaler_path}")
        else:
            logger.warning(f"No scaler found at: {scaler_path}")
            self.scaler = None